In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pickle
import numpy as np
from PIL import Image


In [2]:
### Class LResNet100 for Arcface

class LResNet100(nn.Module):
    def __init__(self):
        super(LResNet100, self).__init__()
        self.conv0 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.bn0 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.relu0 = nn.PReLU(num_parameters=64)
        self.stage1_unit1_bn1 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit1_conv1 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage1_unit1_bn2 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit1_relu1 = nn.PReLU(num_parameters=64)
        self.stage1_unit1_conv2 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage1_unit1_bn3 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit1_conv1sc = nn.Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), dilation=1, bias=False, groups=1, padding=0)
        self.stage1_unit1_sc = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit2_bn1 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit2_conv1 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage1_unit2_bn2 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit2_relu1 = nn.PReLU(num_parameters=64)
        self.stage1_unit2_conv2 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage1_unit2_bn3 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit3_bn1 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit3_conv1 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage1_unit3_bn2 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage1_unit3_relu1 = nn.PReLU(num_parameters=64)
        self.stage1_unit3_conv2 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage1_unit3_bn3 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage2_unit1_bn1 = nn.BatchNorm2d(64, momentum=0.9, eps=2e-05)
        self.stage2_unit1_conv1 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit1_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit1_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit1_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit1_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit1_conv1sc = nn.Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), dilation=1, bias=False, groups=1, padding=0)
        self.stage2_unit1_sc = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit2_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit2_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit2_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit2_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit2_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit2_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit3_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit3_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit3_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit3_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit3_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit3_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit4_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit4_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit4_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit4_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit4_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit4_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit5_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit5_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit5_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit5_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit5_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit5_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit6_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit6_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit6_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit6_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit6_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit6_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit7_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit7_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit7_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit7_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit7_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit7_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit8_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit8_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit8_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit8_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit8_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit8_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit9_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit9_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit9_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit9_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit9_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit9_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit10_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit10_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit10_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit10_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit10_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit10_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit11_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit11_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit11_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit11_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit11_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit11_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit12_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit12_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit12_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit12_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit12_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit12_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit13_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit13_conv1 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit13_bn2 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage2_unit13_relu1 = nn.PReLU(num_parameters=128)
        self.stage2_unit13_conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage2_unit13_bn3 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage3_unit1_bn1 = nn.BatchNorm2d(128, momentum=0.9, eps=2e-05)
        self.stage3_unit1_conv1 = nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit1_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit1_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit1_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit1_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit1_conv1sc = nn.Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), dilation=1, bias=False, groups=1, padding=0)
        self.stage3_unit1_sc = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit2_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit2_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit2_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit2_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit2_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit2_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit3_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit3_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit3_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit3_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit3_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit3_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit4_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit4_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit4_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit4_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit4_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit4_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit5_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit5_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit5_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit5_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit5_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit5_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit6_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit6_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit6_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit6_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit6_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit6_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit7_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit7_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit7_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit7_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit7_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit7_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit8_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit8_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit8_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit8_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit8_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit8_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit9_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit9_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit9_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit9_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit9_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit9_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit10_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit10_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit10_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit10_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit10_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit10_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit11_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit11_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit11_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit11_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit11_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit11_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit12_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit12_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit12_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit12_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit12_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit12_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit13_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit13_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit13_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit13_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit13_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit13_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit14_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit14_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit14_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit14_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit14_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit14_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit15_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit15_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit15_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit15_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit15_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit15_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit16_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit16_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit16_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit16_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit16_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit16_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit17_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit17_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit17_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit17_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit17_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit17_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit18_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit18_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit18_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit18_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit18_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit18_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit19_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit19_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit19_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit19_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit19_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit19_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit20_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit20_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit20_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit20_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit20_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit20_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit21_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit21_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit21_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit21_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit21_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit21_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit22_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit22_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit22_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit22_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit22_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit22_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit23_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit23_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit23_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit23_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit23_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit23_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit24_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit24_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit24_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit24_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit24_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit24_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit25_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit25_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit25_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit25_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit25_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit25_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit26_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit26_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit26_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit26_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit26_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit26_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit27_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit27_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit27_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit27_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit27_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit27_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit28_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit28_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit28_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit28_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit28_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit28_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit29_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit29_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit29_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit29_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit29_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit29_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit30_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit30_conv1 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit30_bn2 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage3_unit30_relu1 = nn.PReLU(num_parameters=256)
        self.stage3_unit30_conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage3_unit30_bn3 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage4_unit1_bn1 = nn.BatchNorm2d(256, momentum=0.9, eps=2e-05)
        self.stage4_unit1_conv1 = nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage4_unit1_bn2 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit1_relu1 = nn.PReLU(num_parameters=512)
        self.stage4_unit1_conv2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage4_unit1_bn3 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit1_conv1sc = nn.Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), dilation=1, bias=False, groups=1, padding=0)
        self.stage4_unit1_sc = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit2_bn1 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit2_conv1 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage4_unit2_bn2 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit2_relu1 = nn.PReLU(num_parameters=512)
        self.stage4_unit2_conv2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage4_unit2_bn3 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit3_bn1 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit3_conv1 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage4_unit3_bn2 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.stage4_unit3_relu1 = nn.PReLU(num_parameters=512)
        self.stage4_unit3_conv2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), dilation=1, bias=False, groups=1, padding=(1, 1))
        self.stage4_unit3_bn3 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.bn1 = nn.BatchNorm2d(512, momentum=0.9, eps=2e-05)
        self.dropout0 = nn.Dropout(p=0.4)
        self.pre_fc1 = nn.Linear(25088, 512, bias=True)
        self.fc1 = nn.BatchNorm1d(512, momentum=0.9, eps=2e-05)

        self.gradients = None
        self.activations = None

    def activations_hook(self, grad):
        self.gradients = grad

    def get_activations_gradient(self):
        return self.gradients

    def get_activations(self):
        return self.activations

    def forward(self, __input__0):
        id = __input__0.clone()
#         _minusscalar0 = -127.5 + id
#         _mulscalar0 = 0.0078125 * _minusscalar0
#         conv0 = self.conv0(2 * (id - 0.5))
        conv0 = self.conv0(id)
        bn0 = self.bn0(conv0)
        relu0 = self.relu0(bn0)
        stage1_unit1_bn1 = self.stage1_unit1_bn1(relu0)
        stage1_unit1_conv1 = self.stage1_unit1_conv1(stage1_unit1_bn1)
        stage1_unit1_bn2 = self.stage1_unit1_bn2(stage1_unit1_conv1)
        stage1_unit1_relu1 = self.stage1_unit1_relu1(stage1_unit1_bn2)
        stage1_unit1_conv2 = self.stage1_unit1_conv2(stage1_unit1_relu1)
        stage1_unit1_bn3 = self.stage1_unit1_bn3(stage1_unit1_conv2)
        stage1_unit1_conv1sc = self.stage1_unit1_conv1sc(relu0)
        stage1_unit1_sc = self.stage1_unit1_sc(stage1_unit1_conv1sc)
        _plus0 = stage1_unit1_bn3 + stage1_unit1_sc
        stage1_unit2_bn1 = self.stage1_unit2_bn1(_plus0)
        stage1_unit2_conv1 = self.stage1_unit2_conv1(stage1_unit2_bn1)
        stage1_unit2_bn2 = self.stage1_unit2_bn2(stage1_unit2_conv1)
        stage1_unit2_relu1 = self.stage1_unit2_relu1(stage1_unit2_bn2)
        stage1_unit2_conv2 = self.stage1_unit2_conv2(stage1_unit2_relu1)
        stage1_unit2_bn3 = self.stage1_unit2_bn3(stage1_unit2_conv2)
        _plus1 = stage1_unit2_bn3 + _plus0
        stage1_unit3_bn1 = self.stage1_unit3_bn1(_plus1)
        stage1_unit3_conv1 = self.stage1_unit3_conv1(stage1_unit3_bn1)
        stage1_unit3_bn2 = self.stage1_unit3_bn2(stage1_unit3_conv1)
        stage1_unit3_relu1 = self.stage1_unit3_relu1(stage1_unit3_bn2)
        stage1_unit3_conv2 = self.stage1_unit3_conv2(stage1_unit3_relu1)
        stage1_unit3_bn3 = self.stage1_unit3_bn3(stage1_unit3_conv2)
        _plus2 = stage1_unit3_bn3 + _plus1
        stage2_unit1_bn1 = self.stage2_unit1_bn1(_plus2)
        stage2_unit1_conv1 = self.stage2_unit1_conv1(stage2_unit1_bn1)
        stage2_unit1_bn2 = self.stage2_unit1_bn2(stage2_unit1_conv1)
        stage2_unit1_relu1 = self.stage2_unit1_relu1(stage2_unit1_bn2)
        stage2_unit1_conv2 = self.stage2_unit1_conv2(stage2_unit1_relu1)
        stage2_unit1_bn3 = self.stage2_unit1_bn3(stage2_unit1_conv2)
        stage2_unit1_conv1sc = self.stage2_unit1_conv1sc(_plus2)
        stage2_unit1_sc = self.stage2_unit1_sc(stage2_unit1_conv1sc)
        _plus3 = stage2_unit1_bn3 + stage2_unit1_sc
        stage2_unit2_bn1 = self.stage2_unit2_bn1(_plus3)
        stage2_unit2_conv1 = self.stage2_unit2_conv1(stage2_unit2_bn1)
        stage2_unit2_bn2 = self.stage2_unit2_bn2(stage2_unit2_conv1)
        stage2_unit2_relu1 = self.stage2_unit2_relu1(stage2_unit2_bn2)
        stage2_unit2_conv2 = self.stage2_unit2_conv2(stage2_unit2_relu1)
        stage2_unit2_bn3 = self.stage2_unit2_bn3(stage2_unit2_conv2)
        _plus4 = stage2_unit2_bn3 + _plus3
        stage2_unit3_bn1 = self.stage2_unit3_bn1(_plus4)
        stage2_unit3_conv1 = self.stage2_unit3_conv1(stage2_unit3_bn1)
        stage2_unit3_bn2 = self.stage2_unit3_bn2(stage2_unit3_conv1)
        stage2_unit3_relu1 = self.stage2_unit3_relu1(stage2_unit3_bn2)
        stage2_unit3_conv2 = self.stage2_unit3_conv2(stage2_unit3_relu1)
        stage2_unit3_bn3 = self.stage2_unit3_bn3(stage2_unit3_conv2)
        _plus5 = stage2_unit3_bn3 + _plus4
        stage2_unit4_bn1 = self.stage2_unit4_bn1(_plus5)
        stage2_unit4_conv1 = self.stage2_unit4_conv1(stage2_unit4_bn1)
        stage2_unit4_bn2 = self.stage2_unit4_bn2(stage2_unit4_conv1)
        stage2_unit4_relu1 = self.stage2_unit4_relu1(stage2_unit4_bn2)
        stage2_unit4_conv2 = self.stage2_unit4_conv2(stage2_unit4_relu1)
        stage2_unit4_bn3 = self.stage2_unit4_bn3(stage2_unit4_conv2)
        _plus6 = stage2_unit4_bn3 + _plus5
        stage2_unit5_bn1 = self.stage2_unit5_bn1(_plus6)
        stage2_unit5_conv1 = self.stage2_unit5_conv1(stage2_unit5_bn1)
        stage2_unit5_bn2 = self.stage2_unit5_bn2(stage2_unit5_conv1)
        stage2_unit5_relu1 = self.stage2_unit5_relu1(stage2_unit5_bn2)
        stage2_unit5_conv2 = self.stage2_unit5_conv2(stage2_unit5_relu1)
        stage2_unit5_bn3 = self.stage2_unit5_bn3(stage2_unit5_conv2)
        _plus7 = stage2_unit5_bn3 + _plus6
        stage2_unit6_bn1 = self.stage2_unit6_bn1(_plus7)
        stage2_unit6_conv1 = self.stage2_unit6_conv1(stage2_unit6_bn1)
        stage2_unit6_bn2 = self.stage2_unit6_bn2(stage2_unit6_conv1)
        stage2_unit6_relu1 = self.stage2_unit6_relu1(stage2_unit6_bn2)
        stage2_unit6_conv2 = self.stage2_unit6_conv2(stage2_unit6_relu1)
        stage2_unit6_bn3 = self.stage2_unit6_bn3(stage2_unit6_conv2)
        _plus8 = stage2_unit6_bn3 + _plus7
        stage2_unit7_bn1 = self.stage2_unit7_bn1(_plus8)
        stage2_unit7_conv1 = self.stage2_unit7_conv1(stage2_unit7_bn1)
        stage2_unit7_bn2 = self.stage2_unit7_bn2(stage2_unit7_conv1)
        stage2_unit7_relu1 = self.stage2_unit7_relu1(stage2_unit7_bn2)
        stage2_unit7_conv2 = self.stage2_unit7_conv2(stage2_unit7_relu1)
        stage2_unit7_bn3 = self.stage2_unit7_bn3(stage2_unit7_conv2)
        _plus9 = stage2_unit7_bn3 + _plus8
        stage2_unit8_bn1 = self.stage2_unit8_bn1(_plus9)
        stage2_unit8_conv1 = self.stage2_unit8_conv1(stage2_unit8_bn1)
        stage2_unit8_bn2 = self.stage2_unit8_bn2(stage2_unit8_conv1)
        stage2_unit8_relu1 = self.stage2_unit8_relu1(stage2_unit8_bn2)
        stage2_unit8_conv2 = self.stage2_unit8_conv2(stage2_unit8_relu1)
        stage2_unit8_bn3 = self.stage2_unit8_bn3(stage2_unit8_conv2)
        _plus10 = stage2_unit8_bn3 + _plus9
        stage2_unit9_bn1 = self.stage2_unit9_bn1(_plus10)
        stage2_unit9_conv1 = self.stage2_unit9_conv1(stage2_unit9_bn1)
        stage2_unit9_bn2 = self.stage2_unit9_bn2(stage2_unit9_conv1)
        stage2_unit9_relu1 = self.stage2_unit9_relu1(stage2_unit9_bn2)
        stage2_unit9_conv2 = self.stage2_unit9_conv2(stage2_unit9_relu1)
        stage2_unit9_bn3 = self.stage2_unit9_bn3(stage2_unit9_conv2)
        _plus11 = stage2_unit9_bn3 + _plus10
        stage2_unit10_bn1 = self.stage2_unit10_bn1(_plus11)
        stage2_unit10_conv1 = self.stage2_unit10_conv1(stage2_unit10_bn1)
        stage2_unit10_bn2 = self.stage2_unit10_bn2(stage2_unit10_conv1)
        stage2_unit10_relu1 = self.stage2_unit10_relu1(stage2_unit10_bn2)
        stage2_unit10_conv2 = self.stage2_unit10_conv2(stage2_unit10_relu1)
        stage2_unit10_bn3 = self.stage2_unit10_bn3(stage2_unit10_conv2)
        _plus12 = stage2_unit10_bn3 + _plus11
        stage2_unit11_bn1 = self.stage2_unit11_bn1(_plus12)
        stage2_unit11_conv1 = self.stage2_unit11_conv1(stage2_unit11_bn1)
        stage2_unit11_bn2 = self.stage2_unit11_bn2(stage2_unit11_conv1)
        stage2_unit11_relu1 = self.stage2_unit11_relu1(stage2_unit11_bn2)
        stage2_unit11_conv2 = self.stage2_unit11_conv2(stage2_unit11_relu1)
        stage2_unit11_bn3 = self.stage2_unit11_bn3(stage2_unit11_conv2)
        _plus13 = stage2_unit11_bn3 + _plus12
        stage2_unit12_bn1 = self.stage2_unit12_bn1(_plus13)
        stage2_unit12_conv1 = self.stage2_unit12_conv1(stage2_unit12_bn1)
        stage2_unit12_bn2 = self.stage2_unit12_bn2(stage2_unit12_conv1)
        stage2_unit12_relu1 = self.stage2_unit12_relu1(stage2_unit12_bn2)
        stage2_unit12_conv2 = self.stage2_unit12_conv2(stage2_unit12_relu1)
        stage2_unit12_bn3 = self.stage2_unit12_bn3(stage2_unit12_conv2)
        _plus14 = stage2_unit12_bn3 + _plus13
        stage2_unit13_bn1 = self.stage2_unit13_bn1(_plus14)
        stage2_unit13_conv1 = self.stage2_unit13_conv1(stage2_unit13_bn1)
        stage2_unit13_bn2 = self.stage2_unit13_bn2(stage2_unit13_conv1)
        stage2_unit13_relu1 = self.stage2_unit13_relu1(stage2_unit13_bn2)
        stage2_unit13_conv2 = self.stage2_unit13_conv2(stage2_unit13_relu1)
        stage2_unit13_bn3 = self.stage2_unit13_bn3(stage2_unit13_conv2)
        _plus15 = stage2_unit13_bn3 + _plus14
        stage3_unit1_bn1 = self.stage3_unit1_bn1(_plus15)
        stage3_unit1_conv1 = self.stage3_unit1_conv1(stage3_unit1_bn1)
        stage3_unit1_bn2 = self.stage3_unit1_bn2(stage3_unit1_conv1)
        stage3_unit1_relu1 = self.stage3_unit1_relu1(stage3_unit1_bn2)
        stage3_unit1_conv2 = self.stage3_unit1_conv2(stage3_unit1_relu1)
        stage3_unit1_bn3 = self.stage3_unit1_bn3(stage3_unit1_conv2)
        stage3_unit1_conv1sc = self.stage3_unit1_conv1sc(_plus15)
        stage3_unit1_sc = self.stage3_unit1_sc(stage3_unit1_conv1sc)
        _plus16 = stage3_unit1_bn3 + stage3_unit1_sc
        stage3_unit2_bn1 = self.stage3_unit2_bn1(_plus16)
        stage3_unit2_conv1 = self.stage3_unit2_conv1(stage3_unit2_bn1)
        stage3_unit2_bn2 = self.stage3_unit2_bn2(stage3_unit2_conv1)
        stage3_unit2_relu1 = self.stage3_unit2_relu1(stage3_unit2_bn2)
        stage3_unit2_conv2 = self.stage3_unit2_conv2(stage3_unit2_relu1)
        stage3_unit2_bn3 = self.stage3_unit2_bn3(stage3_unit2_conv2)
        _plus17 = stage3_unit2_bn3 + _plus16
        stage3_unit3_bn1 = self.stage3_unit3_bn1(_plus17)
        stage3_unit3_conv1 = self.stage3_unit3_conv1(stage3_unit3_bn1)
        stage3_unit3_bn2 = self.stage3_unit3_bn2(stage3_unit3_conv1)
        stage3_unit3_relu1 = self.stage3_unit3_relu1(stage3_unit3_bn2)
        stage3_unit3_conv2 = self.stage3_unit3_conv2(stage3_unit3_relu1)
        stage3_unit3_bn3 = self.stage3_unit3_bn3(stage3_unit3_conv2)
        _plus18 = stage3_unit3_bn3 + _plus17
        stage3_unit4_bn1 = self.stage3_unit4_bn1(_plus18)
        stage3_unit4_conv1 = self.stage3_unit4_conv1(stage3_unit4_bn1)
        stage3_unit4_bn2 = self.stage3_unit4_bn2(stage3_unit4_conv1)
        stage3_unit4_relu1 = self.stage3_unit4_relu1(stage3_unit4_bn2)
        stage3_unit4_conv2 = self.stage3_unit4_conv2(stage3_unit4_relu1)
        stage3_unit4_bn3 = self.stage3_unit4_bn3(stage3_unit4_conv2)
        _plus19 = stage3_unit4_bn3 + _plus18
        stage3_unit5_bn1 = self.stage3_unit5_bn1(_plus19)
        stage3_unit5_conv1 = self.stage3_unit5_conv1(stage3_unit5_bn1)
        stage3_unit5_bn2 = self.stage3_unit5_bn2(stage3_unit5_conv1)
        stage3_unit5_relu1 = self.stage3_unit5_relu1(stage3_unit5_bn2)
        stage3_unit5_conv2 = self.stage3_unit5_conv2(stage3_unit5_relu1)
        stage3_unit5_bn3 = self.stage3_unit5_bn3(stage3_unit5_conv2)
        _plus20 = stage3_unit5_bn3 + _plus19
        stage3_unit6_bn1 = self.stage3_unit6_bn1(_plus20)
        stage3_unit6_conv1 = self.stage3_unit6_conv1(stage3_unit6_bn1)
        stage3_unit6_bn2 = self.stage3_unit6_bn2(stage3_unit6_conv1)
        stage3_unit6_relu1 = self.stage3_unit6_relu1(stage3_unit6_bn2)
        stage3_unit6_conv2 = self.stage3_unit6_conv2(stage3_unit6_relu1)
        stage3_unit6_bn3 = self.stage3_unit6_bn3(stage3_unit6_conv2)
        _plus21 = stage3_unit6_bn3 + _plus20
        stage3_unit7_bn1 = self.stage3_unit7_bn1(_plus21)
        stage3_unit7_conv1 = self.stage3_unit7_conv1(stage3_unit7_bn1)
        stage3_unit7_bn2 = self.stage3_unit7_bn2(stage3_unit7_conv1)
        stage3_unit7_relu1 = self.stage3_unit7_relu1(stage3_unit7_bn2)
        stage3_unit7_conv2 = self.stage3_unit7_conv2(stage3_unit7_relu1)
        stage3_unit7_bn3 = self.stage3_unit7_bn3(stage3_unit7_conv2)
        _plus22 = stage3_unit7_bn3 + _plus21
        stage3_unit8_bn1 = self.stage3_unit8_bn1(_plus22)
        stage3_unit8_conv1 = self.stage3_unit8_conv1(stage3_unit8_bn1)
        stage3_unit8_bn2 = self.stage3_unit8_bn2(stage3_unit8_conv1)
        stage3_unit8_relu1 = self.stage3_unit8_relu1(stage3_unit8_bn2)
        stage3_unit8_conv2 = self.stage3_unit8_conv2(stage3_unit8_relu1)
        stage3_unit8_bn3 = self.stage3_unit8_bn3(stage3_unit8_conv2)
        _plus23 = stage3_unit8_bn3 + _plus22
        stage3_unit9_bn1 = self.stage3_unit9_bn1(_plus23)
        stage3_unit9_conv1 = self.stage3_unit9_conv1(stage3_unit9_bn1)
        stage3_unit9_bn2 = self.stage3_unit9_bn2(stage3_unit9_conv1)
        stage3_unit9_relu1 = self.stage3_unit9_relu1(stage3_unit9_bn2)
        stage3_unit9_conv2 = self.stage3_unit9_conv2(stage3_unit9_relu1)
        stage3_unit9_bn3 = self.stage3_unit9_bn3(stage3_unit9_conv2)
        _plus24 = stage3_unit9_bn3 + _plus23
        stage3_unit10_bn1 = self.stage3_unit10_bn1(_plus24)
        stage3_unit10_conv1 = self.stage3_unit10_conv1(stage3_unit10_bn1)
        stage3_unit10_bn2 = self.stage3_unit10_bn2(stage3_unit10_conv1)
        stage3_unit10_relu1 = self.stage3_unit10_relu1(stage3_unit10_bn2)
        stage3_unit10_conv2 = self.stage3_unit10_conv2(stage3_unit10_relu1)
        stage3_unit10_bn3 = self.stage3_unit10_bn3(stage3_unit10_conv2)
        _plus25 = stage3_unit10_bn3 + _plus24
        stage3_unit11_bn1 = self.stage3_unit11_bn1(_plus25)
        stage3_unit11_conv1 = self.stage3_unit11_conv1(stage3_unit11_bn1)
        stage3_unit11_bn2 = self.stage3_unit11_bn2(stage3_unit11_conv1)
        stage3_unit11_relu1 = self.stage3_unit11_relu1(stage3_unit11_bn2)
        stage3_unit11_conv2 = self.stage3_unit11_conv2(stage3_unit11_relu1)
        stage3_unit11_bn3 = self.stage3_unit11_bn3(stage3_unit11_conv2)
        _plus26 = stage3_unit11_bn3 + _plus25
        stage3_unit12_bn1 = self.stage3_unit12_bn1(_plus26)
        stage3_unit12_conv1 = self.stage3_unit12_conv1(stage3_unit12_bn1)
        stage3_unit12_bn2 = self.stage3_unit12_bn2(stage3_unit12_conv1)
        stage3_unit12_relu1 = self.stage3_unit12_relu1(stage3_unit12_bn2)
        stage3_unit12_conv2 = self.stage3_unit12_conv2(stage3_unit12_relu1)
        stage3_unit12_bn3 = self.stage3_unit12_bn3(stage3_unit12_conv2)
        _plus27 = stage3_unit12_bn3 + _plus26
        stage3_unit13_bn1 = self.stage3_unit13_bn1(_plus27)
        stage3_unit13_conv1 = self.stage3_unit13_conv1(stage3_unit13_bn1)
        stage3_unit13_bn2 = self.stage3_unit13_bn2(stage3_unit13_conv1)
        stage3_unit13_relu1 = self.stage3_unit13_relu1(stage3_unit13_bn2)
        stage3_unit13_conv2 = self.stage3_unit13_conv2(stage3_unit13_relu1)
        stage3_unit13_bn3 = self.stage3_unit13_bn3(stage3_unit13_conv2)
        _plus28 = stage3_unit13_bn3 + _plus27
        stage3_unit14_bn1 = self.stage3_unit14_bn1(_plus28)
        stage3_unit14_conv1 = self.stage3_unit14_conv1(stage3_unit14_bn1)
        stage3_unit14_bn2 = self.stage3_unit14_bn2(stage3_unit14_conv1)
        stage3_unit14_relu1 = self.stage3_unit14_relu1(stage3_unit14_bn2)
        stage3_unit14_conv2 = self.stage3_unit14_conv2(stage3_unit14_relu1)
        stage3_unit14_bn3 = self.stage3_unit14_bn3(stage3_unit14_conv2)
        _plus29 = stage3_unit14_bn3 + _plus28
        stage3_unit15_bn1 = self.stage3_unit15_bn1(_plus29)
        stage3_unit15_conv1 = self.stage3_unit15_conv1(stage3_unit15_bn1)
        stage3_unit15_bn2 = self.stage3_unit15_bn2(stage3_unit15_conv1)
        stage3_unit15_relu1 = self.stage3_unit15_relu1(stage3_unit15_bn2)
        stage3_unit15_conv2 = self.stage3_unit15_conv2(stage3_unit15_relu1)
        stage3_unit15_bn3 = self.stage3_unit15_bn3(stage3_unit15_conv2)
        _plus30 = stage3_unit15_bn3 + _plus29
        stage3_unit16_bn1 = self.stage3_unit16_bn1(_plus30)
        stage3_unit16_conv1 = self.stage3_unit16_conv1(stage3_unit16_bn1)
        stage3_unit16_bn2 = self.stage3_unit16_bn2(stage3_unit16_conv1)
        stage3_unit16_relu1 = self.stage3_unit16_relu1(stage3_unit16_bn2)
        stage3_unit16_conv2 = self.stage3_unit16_conv2(stage3_unit16_relu1)
        stage3_unit16_bn3 = self.stage3_unit16_bn3(stage3_unit16_conv2)
        _plus31 = stage3_unit16_bn3 + _plus30
        stage3_unit17_bn1 = self.stage3_unit17_bn1(_plus31)
        stage3_unit17_conv1 = self.stage3_unit17_conv1(stage3_unit17_bn1)
        stage3_unit17_bn2 = self.stage3_unit17_bn2(stage3_unit17_conv1)
        stage3_unit17_relu1 = self.stage3_unit17_relu1(stage3_unit17_bn2)
        stage3_unit17_conv2 = self.stage3_unit17_conv2(stage3_unit17_relu1)
        stage3_unit17_bn3 = self.stage3_unit17_bn3(stage3_unit17_conv2)
        _plus32 = stage3_unit17_bn3 + _plus31
        stage3_unit18_bn1 = self.stage3_unit18_bn1(_plus32)
        stage3_unit18_conv1 = self.stage3_unit18_conv1(stage3_unit18_bn1)
        stage3_unit18_bn2 = self.stage3_unit18_bn2(stage3_unit18_conv1)
        stage3_unit18_relu1 = self.stage3_unit18_relu1(stage3_unit18_bn2)
        stage3_unit18_conv2 = self.stage3_unit18_conv2(stage3_unit18_relu1)
        stage3_unit18_bn3 = self.stage3_unit18_bn3(stage3_unit18_conv2)
        _plus33 = stage3_unit18_bn3 + _plus32
        stage3_unit19_bn1 = self.stage3_unit19_bn1(_plus33)
        stage3_unit19_conv1 = self.stage3_unit19_conv1(stage3_unit19_bn1)
        stage3_unit19_bn2 = self.stage3_unit19_bn2(stage3_unit19_conv1)
        stage3_unit19_relu1 = self.stage3_unit19_relu1(stage3_unit19_bn2)
        stage3_unit19_conv2 = self.stage3_unit19_conv2(stage3_unit19_relu1)
        stage3_unit19_bn3 = self.stage3_unit19_bn3(stage3_unit19_conv2)
        _plus34 = stage3_unit19_bn3 + _plus33
        stage3_unit20_bn1 = self.stage3_unit20_bn1(_plus34)
        stage3_unit20_conv1 = self.stage3_unit20_conv1(stage3_unit20_bn1)
        stage3_unit20_bn2 = self.stage3_unit20_bn2(stage3_unit20_conv1)
        stage3_unit20_relu1 = self.stage3_unit20_relu1(stage3_unit20_bn2)
        stage3_unit20_conv2 = self.stage3_unit20_conv2(stage3_unit20_relu1)
        stage3_unit20_bn3 = self.stage3_unit20_bn3(stage3_unit20_conv2)
        _plus35 = stage3_unit20_bn3 + _plus34
        stage3_unit21_bn1 = self.stage3_unit21_bn1(_plus35)
        stage3_unit21_conv1 = self.stage3_unit21_conv1(stage3_unit21_bn1)
        stage3_unit21_bn2 = self.stage3_unit21_bn2(stage3_unit21_conv1)
        stage3_unit21_relu1 = self.stage3_unit21_relu1(stage3_unit21_bn2)
        stage3_unit21_conv2 = self.stage3_unit21_conv2(stage3_unit21_relu1)
        stage3_unit21_bn3 = self.stage3_unit21_bn3(stage3_unit21_conv2)
        _plus36 = stage3_unit21_bn3 + _plus35
        stage3_unit22_bn1 = self.stage3_unit22_bn1(_plus36)
        stage3_unit22_conv1 = self.stage3_unit22_conv1(stage3_unit22_bn1)
        stage3_unit22_bn2 = self.stage3_unit22_bn2(stage3_unit22_conv1)
        stage3_unit22_relu1 = self.stage3_unit22_relu1(stage3_unit22_bn2)
        stage3_unit22_conv2 = self.stage3_unit22_conv2(stage3_unit22_relu1)
        stage3_unit22_bn3 = self.stage3_unit22_bn3(stage3_unit22_conv2)
        _plus37 = stage3_unit22_bn3 + _plus36
        stage3_unit23_bn1 = self.stage3_unit23_bn1(_plus37)
        stage3_unit23_conv1 = self.stage3_unit23_conv1(stage3_unit23_bn1)
        stage3_unit23_bn2 = self.stage3_unit23_bn2(stage3_unit23_conv1)
        stage3_unit23_relu1 = self.stage3_unit23_relu1(stage3_unit23_bn2)
        stage3_unit23_conv2 = self.stage3_unit23_conv2(stage3_unit23_relu1)
        stage3_unit23_bn3 = self.stage3_unit23_bn3(stage3_unit23_conv2)
        _plus38 = stage3_unit23_bn3 + _plus37
        stage3_unit24_bn1 = self.stage3_unit24_bn1(_plus38)
        stage3_unit24_conv1 = self.stage3_unit24_conv1(stage3_unit24_bn1)
        stage3_unit24_bn2 = self.stage3_unit24_bn2(stage3_unit24_conv1)
        stage3_unit24_relu1 = self.stage3_unit24_relu1(stage3_unit24_bn2)
        stage3_unit24_conv2 = self.stage3_unit24_conv2(stage3_unit24_relu1)
        stage3_unit24_bn3 = self.stage3_unit24_bn3(stage3_unit24_conv2)
        _plus39 = stage3_unit24_bn3 + _plus38
        stage3_unit25_bn1 = self.stage3_unit25_bn1(_plus39)
        stage3_unit25_conv1 = self.stage3_unit25_conv1(stage3_unit25_bn1)
        stage3_unit25_bn2 = self.stage3_unit25_bn2(stage3_unit25_conv1)
        stage3_unit25_relu1 = self.stage3_unit25_relu1(stage3_unit25_bn2)
        stage3_unit25_conv2 = self.stage3_unit25_conv2(stage3_unit25_relu1)
        stage3_unit25_bn3 = self.stage3_unit25_bn3(stage3_unit25_conv2)
        _plus40 = stage3_unit25_bn3 + _plus39
        stage3_unit26_bn1 = self.stage3_unit26_bn1(_plus40)
        stage3_unit26_conv1 = self.stage3_unit26_conv1(stage3_unit26_bn1)
        stage3_unit26_bn2 = self.stage3_unit26_bn2(stage3_unit26_conv1)
        stage3_unit26_relu1 = self.stage3_unit26_relu1(stage3_unit26_bn2)
        stage3_unit26_conv2 = self.stage3_unit26_conv2(stage3_unit26_relu1)
        stage3_unit26_bn3 = self.stage3_unit26_bn3(stage3_unit26_conv2)
        _plus41 = stage3_unit26_bn3 + _plus40
        stage3_unit27_bn1 = self.stage3_unit27_bn1(_plus41)
        stage3_unit27_conv1 = self.stage3_unit27_conv1(stage3_unit27_bn1)
        stage3_unit27_bn2 = self.stage3_unit27_bn2(stage3_unit27_conv1)
        stage3_unit27_relu1 = self.stage3_unit27_relu1(stage3_unit27_bn2)
        stage3_unit27_conv2 = self.stage3_unit27_conv2(stage3_unit27_relu1)
        stage3_unit27_bn3 = self.stage3_unit27_bn3(stage3_unit27_conv2)
        _plus42 = stage3_unit27_bn3 + _plus41
        stage3_unit28_bn1 = self.stage3_unit28_bn1(_plus42)
        stage3_unit28_conv1 = self.stage3_unit28_conv1(stage3_unit28_bn1)
        stage3_unit28_bn2 = self.stage3_unit28_bn2(stage3_unit28_conv1)
        stage3_unit28_relu1 = self.stage3_unit28_relu1(stage3_unit28_bn2)
        stage3_unit28_conv2 = self.stage3_unit28_conv2(stage3_unit28_relu1)
        stage3_unit28_bn3 = self.stage3_unit28_bn3(stage3_unit28_conv2)
        _plus43 = stage3_unit28_bn3 + _plus42
        stage3_unit29_bn1 = self.stage3_unit29_bn1(_plus43)
        stage3_unit29_conv1 = self.stage3_unit29_conv1(stage3_unit29_bn1)
        stage3_unit29_bn2 = self.stage3_unit29_bn2(stage3_unit29_conv1)
        stage3_unit29_relu1 = self.stage3_unit29_relu1(stage3_unit29_bn2)
        stage3_unit29_conv2 = self.stage3_unit29_conv2(stage3_unit29_relu1)
        stage3_unit29_bn3 = self.stage3_unit29_bn3(stage3_unit29_conv2)
        _plus44 = stage3_unit29_bn3 + _plus43
        stage3_unit30_bn1 = self.stage3_unit30_bn1(_plus44)
        stage3_unit30_conv1 = self.stage3_unit30_conv1(stage3_unit30_bn1)
        stage3_unit30_bn2 = self.stage3_unit30_bn2(stage3_unit30_conv1)
        stage3_unit30_relu1 = self.stage3_unit30_relu1(stage3_unit30_bn2)
        stage3_unit30_conv2 = self.stage3_unit30_conv2(stage3_unit30_relu1)
        stage3_unit30_bn3 = self.stage3_unit30_bn3(stage3_unit30_conv2)
        _plus45 = stage3_unit30_bn3 + _plus44
        stage4_unit1_bn1 = self.stage4_unit1_bn1(_plus45)
        stage4_unit1_conv1 = self.stage4_unit1_conv1(stage4_unit1_bn1)
        stage4_unit1_bn2 = self.stage4_unit1_bn2(stage4_unit1_conv1)
        stage4_unit1_relu1 = self.stage4_unit1_relu1(stage4_unit1_bn2)
        stage4_unit1_conv2 = self.stage4_unit1_conv2(stage4_unit1_relu1)
        stage4_unit1_bn3 = self.stage4_unit1_bn3(stage4_unit1_conv2)
        stage4_unit1_conv1sc = self.stage4_unit1_conv1sc(_plus45)
        stage4_unit1_sc = self.stage4_unit1_sc(stage4_unit1_conv1sc)
        _plus46 = stage4_unit1_bn3 + stage4_unit1_sc
        stage4_unit2_bn1 = self.stage4_unit2_bn1(_plus46)
        stage4_unit2_conv1 = self.stage4_unit2_conv1(stage4_unit2_bn1)
        stage4_unit2_bn2 = self.stage4_unit2_bn2(stage4_unit2_conv1)
        stage4_unit2_relu1 = self.stage4_unit2_relu1(stage4_unit2_bn2)
        stage4_unit2_conv2 = self.stage4_unit2_conv2(stage4_unit2_relu1)
        stage4_unit2_bn3 = self.stage4_unit2_bn3(stage4_unit2_conv2)
        _plus47 = stage4_unit2_bn3 + _plus46
        stage4_unit3_bn1 = self.stage4_unit3_bn1(_plus47)
        stage4_unit3_conv1 = self.stage4_unit3_conv1(stage4_unit3_bn1)
        stage4_unit3_bn2 = self.stage4_unit3_bn2(stage4_unit3_conv1)
        stage4_unit3_relu1 = self.stage4_unit3_relu1(stage4_unit3_bn2)
        stage4_unit3_conv2 = self.stage4_unit3_conv2(stage4_unit3_relu1)

        ### grad-cam
        # h = stage4_unit3_conv2.register_hook(self.activations_hook)
        # self.activations = stage4_unit3_conv2

        stage4_unit3_bn3 = self.stage4_unit3_bn3(stage4_unit3_conv2)
        _plus48 = stage4_unit3_bn3 + _plus47
        bn1 = self.bn1(_plus48)
        dropout0 = self.dropout0(bn1)

        pre_fc1 = self.pre_fc1(dropout0.view([int(dropout0.size(0)), -1]))


        fc1 = self.fc1(pre_fc1)
        return fc1


In [29]:
if __name__ == '__main__':

    net = LResNet100()
    net.load_state_dict(torch.load('torchtest.pt'))
    net.eval()

    filename = 'FFHQ/00000.png'

    img1 = np.array(Image.open(filename)).astype(np.float32)
    img1.resize((112, 112, 3))
    img1 = np.transpose(img1, (2, 0, 1))
    img1 = img1.reshape(1, 3, 112, 112)
    
    img1t = torch.tensor(img1)/255.0
    print(img1t)
    out1t = net(img1t).cpu().detach().numpy().flatten()

    print(out1t)

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0039],
          [0.0078, 0.0118, 0.0353,  ..., 0.1216, 0.1725, 0.3333],
          ...,
          [0.4000, 0.4000, 0.4039,  ..., 0.5765, 0.4902, 0.4078],
          [0.4157, 0.4235, 0.4118,  ..., 0.6784, 0.6745, 0.6588],
          [0.6588, 0.6549, 0.6471,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.5098, 0.5020, 0.4902,  ..., 0.5294, 0.5216, 0.5059],
          [0.5059, 0.4980, 0.4784,  ..., 0.8078, 0.7882, 0.7529],
          [0.7412, 0.7373, 0.7333,  ..., 0.9961, 1.0000, 1.0000],
          ...,
          [0.3529, 0.3569, 0.3490,  ..., 0.5451, 0.4314, 0.3255],
          [0.3294, 0.3569, 0.3529,  ..., 0.8980, 0.9020, 0.8863],
          [0.8824, 0.8745, 0.8745,  ..., 0.4392, 0.4471, 0.4667]],

         [[0.5725, 0.5647, 0.5529,  ..., 0.6745, 0.6667, 0.6510],
          [0.6392, 0.6314, 0.6157,  ..., 0.9176, 0.9137, 0.8980],
          [0.8824, 0.8784, 0.8745,  ..., 0